In [ ]:
!pip install timm

In [ ]:
import timm
import pprint
model_pretrain_list = timm.list_models(pretrained=True)
# for i in model_pretrain_list:
#     print(i, '\n')

In [ ]:
import pandas as pd
import numpy as np
import random

import time
import torchvision
import torch.nn as nn
from tqdm import tqdm_notebook as tqdm

from PIL import Image, ImageFile
from torch.utils.data import Dataset
import torch
import torch.optim as optim
from torchvision import transforms
from torch.optim import lr_scheduler
import os

device = torch.device("cuda:0")
ImageFile.LOAD_TRUNCATED_IMAGES = True


os.environ['CUDA_VISIBLE_DEVICE'] = '0'

torch.cuda.set_device(0)
modellr = 1e-5
BATCH_SIZE = 64
EPOCHS = 100
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
def setup_seed(seed):
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
setup_seed(2333)

In [ ]:
class RetinopathyDatasetTrain(Dataset):

    def __init__(self, csv_file):

        self.data = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join('../input/aptos2019-blindness-detection/train_images', self.data.loc[idx, 'id_code'] + '.png')
        image = Image.open(img_name)
        image = image.resize((224, 224), resample=Image.BILINEAR)
        label = torch.tensor(self.data.loc[idx, 'diagnosis'])
        return {'image': transforms.ToTensor()(image),
                'labels': label
                }
    
class RetinopathyDatasetTest(Dataset):

    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        img_name = os.path.join('../input/aptos2019-blindness-detection/test_images', self.data.loc[idx, 'id_code'] + '.png')
        image = Image.open(img_name)
        image = image.resize((224, 224), resample=Image.BILINEAR)
        return transforms.ToTensor()(image)

In [ ]:
import timm
model = timm.create_model('resnest50d', pretrained=True, num_classes=5)

# model = timm.create_model('resnest101e', pretrained=True, num_classes=5)

model.load_state_dict(torch.load('../input/aptos-resnet-ckp/ckp (2).pt'))
model = model.to(device)

In [ ]:
criterion = nn.CrossEntropyLoss()

optimizer = optim.Adam(model.parameters(), lr=modellr)

# scheduler = optim.WarmupCosineSchedule(optimizer, warmup_steps=50, t_total=200)

def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    modellrnew = modellr * (0.1 ** (epoch // 50))
    print("lr:", modellrnew)
    for param_group in optimizer.param_groups:
        param_group['lr'] = modellrnew

In [ ]:
dataset = RetinopathyDatasetTrain(csv_file='../input/aptos2019-blindness-detection/train.csv')
train_dataset, valid_dataset = torch.utils.data.random_split(dataset, [int(0.8 * len(dataset)), len(dataset) - int(0.8*len(dataset))])
data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=False)
valid_data_loader = torch.utils.data.DataLoader(valid_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, drop_last=False)

In [ ]:
best_acc = 0
train_loss, valid_loss, valid_acc = [], [], []
train_loss = np.load('../input/aptos-resnet-ckp/train_loss (2).npy')
valid_loss = np.load('../input/aptos-resnet-ckp/valid_loss (2).npy')
valid_acc = np.load('../input/aptos-resnet-ckp/valid_acc (2).npy')
valid_loss = valid_loss.tolist()
train_loss = train_loss.tolist()
valid_acc = valid_acc.tolist()

In [ ]:
@torch.no_grad()
def Valid():
    print('Valid')
    tk1 = tqdm(valid_data_loader, total=int(len(valid_data_loader)))
    acc_num, sum_num, loss_sum = 0, 0, 0
    for bi, d in enumerate(tk1):
        inputs = d["image"]
        labels = d["labels"]
        inputs = inputs.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.long)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        acc_num += (labels==outputs.argmax(-1)).sum()
        sum_num += labels.size(0)
        loss_sum += loss.item() * inputs.size(0)
    acc = acc_num / sum_num
    print('Valid Acc: {:.4f}'.format(acc))
    global best_acc
    if acc > best_acc:
        best_acc = acc
        torch.save(model.state_dict(), 'best_model.pt')
        print('save best model with acc:{:.4f}'.format(acc))
    valid_loss.append(loss_sum / len(valid_data_loader))
    valid_acc.append(acc.cpu())

In [ ]:
import matplotlib.pyplot as plt
def plotTheCurve():
    plt.figure(figsize=(8,6))
    plt.plot([i for i in range(len(train_loss))],train_loss,'',label="train_loss")
    plt.plot([i for i in range(len(valid_loss))],valid_loss,'',label="valid_loss")
    plt.title('loss')
    plt.legend(loc='upper right')
    plt.xlabel('epoch')
    plt.ylabel('')
    plt.grid(len(train_loss))
    plt.show()
    
    plt.figure(figsize=(8,6))
    plt.plot([i for i in range(len(valid_acc))],valid_acc,'',label="acc")

    plt.title('acc')
    plt.legend(loc='upper right')
    plt.xlabel('epoch')
    plt.ylabel('')
    plt.grid(len(train_loss))
    plt.show()

In [ ]:

since = time.time()
for epoch in range(0, EPOCHS):
    print('Epoch {}/{}'.format(epoch, EPOCHS - 1))
    print('-' * 30)
    adjust_learning_rate(optimizer, epoch)
#     print("lr:", scheduler.get_lr()[0])
    model.train()
    running_loss = 0.0
    tk0 = tqdm(data_loader, total=int(len(data_loader)))
    counter = 0
    for bi, d in enumerate(tk0):
        inputs = d["image"]
        labels = d["labels"]
        inputs = inputs.to(device, dtype=torch.float)
        labels = labels.to(device, dtype=torch.long)
        optimizer.zero_grad()
        with torch.set_grad_enabled(True):
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
#             scheduler.step()
            optimizer.step()
        running_loss += loss.item() * inputs.size(0)
        counter += 1
        tk0.set_postfix(loss=(running_loss / (counter * data_loader.batch_size)))
    epoch_loss = running_loss / len(data_loader)
    print('Training Loss: {:.4f}'.format(epoch_loss))
    Valid()
    train_loss.append(epoch_loss)
    plotTheCurve()
    

time_elapsed = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(time_elapsed // 60, time_elapsed % 60))
torch.save(model.state_dict(), "model.bin")

In [ ]:

torch.save(model.state_dict(), './ckp.pt')

In [ ]:
print(best_acc)
print(valid_acc[-1])

In [ ]:
print(type(valid_acc[1])==float)
# valid_acc = np.array(valid_acc)
valid_acc = [i if type(i)==float else i.cpu() for i in valid_acc]

In [ ]:
import matplotlib.pyplot as plt
plt.figure(figsize=(8,6))
plt.plot([i for i in range(len(train_loss))],train_loss,'',label="train_loss")
plt.plot([i for i in range(len(valid_loss))],valid_loss,'',label="valid_loss")
plt.title('loss')
plt.legend(loc='upper right')
plt.xlabel('epoch')
plt.ylabel('')
plt.grid(len(train_loss))
plt.show()

In [ ]:
plt.figure(figsize=(8,6))
plt.plot([i for i in range(len(valid_acc))],valid_acc,'',label="acc")

plt.title('acc')
plt.legend(loc='upper right')
plt.xlabel('epoch')
plt.ylabel('')
plt.grid(len(train_loss))
plt.show()

In [ ]:
np.save('train_loss.npy', train_loss)
np.save('valid_loss.npy', valid_loss)
np.save('valid_acc.npy', valid_acc)

In [ ]:
import os
os.chdir('/kaggle/working')
print(os.getcwd())
print(os.listdir("/kaggle/working"))
from IPython.display import FileLink
for i in os.listdir("/kaggle/working"):
    FileLink(i)
FileLink('ckp.pt')